In [1]:
import os 
#import audioop
import subprocess
import speech_recognition as sr
#from at16k.api import SpeechToText
from auditok import split, AudioRegion

### 1. Online Transcription (Google API)

In [4]:
def CreateSentences(path): 
    
    #PreProcessing
    Audio = AudioRegion.load(path) 

    Sentences = open("recognized.txt", "w+") 
    
    Data = audioop.tomono(bytes(Audio), Audio.sample_width, 0.5, 0.5)
    
    region_mono = AudioRegion(Data, sample_width=Audio.sw, sampling_rate=Audio.sr, channels=1)
    
    regions = region_mono.split(energy_threshold=50, use_channel='mix', min_dur=1, max_dur=15, max_silence = 0.25) 
    
    #Extract pauses
    onset = 0
    pauses = []
    speeches, NewSpeeches = [], []
    for r in regions:
        if onset < r.meta.start:
            pauses.append((onset, r.meta.start))
            onset = r.meta.end
        speeches.append(r)

    #Last pause if there exists
    if onset < Audio.duration:
        pauses.append((onset, Audio.duration))

    silence = []
    for pause in pauses:
        silence.append(Audio.seconds[pause[0]:pause[1]])
        
    #Setting 15 Sec limitation
    i = 0
    max_limit = 15
    while i < len(speeches):
        sound = speeches[i]
        while i < len(speeches)-1 and sound.duration + speeches[i+1].duration < max_limit:
            i += 1
            sound += speeches[i]
        else:
            i+=1
        NewSpeeches.append(sound)

  
    i = 0
    #Speech Recognition
    print('Found {0} Audio Sentences: '.format(len(NewSpeeches)))
    for speech in NewSpeeches: 
      
        print("Processing Speech " + str(i), end="\r") 
    
        r = sr.Recognizer() 

        audio_listened = sr.AudioData(bytes(NewSpeeches[i]), NewSpeeches[i].sr, NewSpeeches[i].sample_width)
  
        try: 
            rec = r.recognize_google(audio_listened) 
            Sentences.write(rec+". ") 
  
        except sr.UnknownValueError: 
            print("Could not understand audio") 
  
        except sr.RequestError as e: 
            print("Could not request results. check your internet connection") 
  
        i += 1
  
    Sentences.close()
    print('Audio Sentences processed: ', len(NewSpeeches))

In [21]:
print('Enter the audio file path') 
path = input()
CreateSentences(path)

Enter the audio file path
ng.wav
Found 33 Audio Sentences: 
Audio Sentences processed:  33


### 1. Offline Transcription (at16k Model)

In [7]:
def xCreateSentences(path): 
    
    #PreProcessing
    Convert = SpeechToText('en_8k')
    
    subprocess.call(['ffmpeg', '-i', path, '-ar', '8000', '-ac', '1', '-ab', '16', 'Output.wav'])

    Audio = AudioRegion.load('Output.wav') 

    Sentences = open("recognized.txt", "w+") 
    
    regions = Audio.split(energy_threshold=50, use_channel='mix', min_dur=1, max_dur=15, max_silence = 0.25) 
    
    #Extract pauses
    onset = 0
    pauses = []
    speeches, NewSpeeches = [], []
    for r in regions:
        if onset < r.meta.start:
            pauses.append((onset, r.meta.start))
            onset = r.meta.end
        speeches.append(r)

    #Last pause if there exists
    if onset < Audio.duration:
        pauses.append((onset, Audio.duration))

    silence = []
    for pause in pauses:
        silence.append(Audio.seconds[pause[0]:pause[1]])
        
    #Setting 15 Sec limitation
    i = 0
    max_limit = 15
    while i < len(speeches):
        sound = speeches[i]
        while i < len(speeches)-1 and sound.duration + speeches[i+1].duration < max_limit:
            i += 1
            sound += speeches[i]
        else:
            i+=1
        NewSpeeches.append(sound)

  
    i = 0
    #Speech Recognition
    print('Found {0} Audio Sentences: '.format(len(NewSpeeches)))
    for speech in NewSpeeches: 
      
        print("Processing Speech " + str(i), end="\r") 
        
        speech.save('Speech '+str(i)+'.wav')
        
        Text = Convert('Speech '+str(i)+'.wav')
 
        rec = Text['text']
    
        Sentences.write(rec+". ") 
        
        os.remove('Speech '+str(i)+'.wav')
  
        i += 1
    os.remove('Output.wav')
    Sentences.close()
    print('Audio Sentences processed: ', len(NewSpeeches))

In [8]:
print('Enter the audio file path') 
path = str(input())
xCreateSentences(path)

Enter the audio file path
ng.wav
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\khled\.at16k\en_8k\export\1569649891\variables\variables
Found 31 Audio Sentences: 
Audio Sentences processed:  31
